In [1]:
using Pkg; Pkg.activate("../../../FermiCG/")
using FermiCG
using PyCall
using Plots
using LinearAlgebra
using Printf
using JLD2

  Activating project at `~/code/FermiCG`


In [2]:
#molecule = "
#He 0.00000000 0.00000000 0.00000000
#He 1.41421356 0.00000000 0.00000000
#He 0.00000000 1.41421356 0.00000000
#He 1.41421356 1.41421356 0.00000000
#He 0.70710678 0.70710678 1.00000000
#He 0.70710678 0.70710678 -1.00000000
#"
#molecule = "
#He 0.00000000 0.00000000 0.00000000
#He 2.12132035 0.00000000 0.00000000
#He 0.00000000 2.12132035 0.00000000
#He 2.12132034 2.12132034 0.00000000
#He 1.06066017 1.06066017 1.50000000
#He 1.06066017 1.06066017 -1.50000000
#"
molecule = "
He       0.0000000000000000       0.0000000000000000       0.0000000000000000
He       3.8890872900000004       0.0000000000000000       0.0000000000000000
He       0.0000000000000000       3.8890872900000004       0.0000000000000000
He       3.8890872900000004       3.8890872900000004       0.0000000000000000
He       1.9445436450000002       1.9445436450000002       2.7500000000000000
He       1.9445436450000002       1.9445436450000002      -2.7500000000000000
"
atoms = []
for (li,line) in enumerate(split(rstrip(lstrip(molecule)), "\n"))
    l = split(line)
    push!(atoms, Atom(li, l[1], parse.(Float64,l[2:4])))
end

basis = "aug-cc-pvdz"

# Create FermiCG.Molecule type
mol     = Molecule(0, 1, atoms,basis);

pyscf = pyimport("pyscf")

n_steps = 35
step_size = .05

pymol_init = pyscf.gto.Mole(atom=molecule,
                            symmetry = false, spin =0,charge=0,
                            basis = basis)
pymol_init.build()

PyObject <pyscf.gto.mole.Mole object at 0x7fd7c83ba690>

In [6]:
io = open("traj.xyz", "w");
energies_ground = []
energies_t1 = []
energies_t2 = []
energies_t3 = []
energies_t4 = []
energies_t5 = []
energies_t6 = []

energies_s1 = []
energies_s2 = []
energies_s3 = []
energies_s4 = []
energies_s5 = []
energies_s6 = []

#energies_t7 = []
#energies_t8 = []
#energies_t9 = []
#energies_t10 = []
#energies_t11 = []
#energies_t12 = []
pt2_energies = []

lo = pyimport("pyscf.lo.orth")
tools = pyimport("pyscf.tools")
fcidump = pyimport("pyscf.tools.fcidump");

In [12]:
mf = pyscf.scf.RHF(pymol_init).run()
    
s = mf.get_ovlp(pymol_init)

lo_ao = lo.lowdin(s)
println("size of Lowdin ortho AO's:", size(lo_ao))

#write fci dump file from the modified mo coefficients
tools.fcidump.from_mo(pymol_init, "fcidump.he06_oct", lo_ao)

#Can just read in pyscf dump file for integrals (once you have already run an scf calculation)
ctx = fcidump.read("fcidump.he06_oct");
h = ctx["H1"];
g = ctx["H2"];
ecore = ctx["ECORE"];
g = pyscf.ao2mo.restore("1", g, size(h,2))

#This one below was not working. Error: setfield! immutable struct of type InCoreInts cannot be changed
ints = InCoreInts(ecore,h,g);
Da_old = zeros(size(ints.h1))
Db_old = zeros(size(ints.h1))

C_old = lo_ao

for R in 1:n_steps

    pymol = deepcopy(pymol_init)
    scale = 1+R*step_size
    
    #move to smaller geometry
    xyz = @sprintf("%5i\n\n", length(mol.atoms))
    tmp = []
    for a in mol.atoms
        push!(tmp, ["He", (a.xyz[1]/scale, a.xyz[2]/scale, a.xyz[3]/scale)])
        xyz = xyz * @sprintf("%6s %24.16f %24.16f %24.16f \n", a.symbol, a.xyz[1]/scale, a.xyz[2]/scale, a.xyz[3]/scale)
    end
    
    #move to larger geometry
    #for a in mol.atoms
    #    push!(tmp, ["He", (a.xyz[1]*scale, a.xyz[2]*scale, a.xyz[3]*scale)])
    #    xyz = xyz * @sprintf("%6s %24.16f %24.16f %24.16f \n", a.symbol, a.xyz[1]*scale, a.xyz[2]*scale, a.xyz[3]*scale)
    #end
    #pymol.atom = tmp
    #pymol.build()


    println(xyz)
    write(io, xyz);


    #println(pymol.format_atom(1))

    #     mol_R = Molecule(0, 1, [a[0]pymol.atom, pymol.basis)


    mf = pyscf.scf.RHF(pymol).run()
    
    s = mf.get_ovlp(pymol)

    lo_ao = lo.lowdin(s)
    println("size of Lowdin ortho AO's:", size(lo_ao))

    #write fci dump file from the modified mo coefficients
    tools.fcidump.from_mo(pymol, "fcidump.he06_oct", lo_ao)

    #Can just read in pyscf dump file for integrals (once you have already run an scf calculation)
    ctx = fcidump.read("fcidump.he06_oct");
    h = ctx["H1"];
    g = ctx["H2"];
    ecore = ctx["ECORE"];
    g = pyscf.ao2mo.restore("1", g, size(h,2))

    #This one below was not working. Error: setfield! immutable struct of type InCoreInts cannot be changed
    ints = InCoreInts(ecore,h,g);

    #Run cmf
    #Define clusters and intial Fock space for inital CMF calc for 14 orbs each He
    #clusters_in = [(1:14),(15:28), (29:42), (43:56), (57:70), (71:84), (85:98)]

    #Define clusters and intial Fock space for inital CMF calc for 9 orbs each He
    clusters_in = [(1:9),(10:18), (19:27), (28:36), (37:45), (46:54)]

    #Define clusters and intial Fock space for inital CMF calc for 5 orbs each He
    #clusters_in = [(1:5),(6:10), (11:15), (16:20), (21:25), (26:30)]
    init_fspace = [(1,1),(1,1),(1,1),(1,1),(1,1),(1,1)]
    rdm1 = zeros(size(ints.h1))
    na=6
    nb=6

    #Define clusters now using FermiCG code
    clusters = [Cluster(i,collect(clusters_in[i])) for i = 1:length(clusters_in)]
    display(clusters)

    #@save "before_cmf.jld2" ints clusters init_fspace

    print(size(ints.h1))
        
    #do a CMF calculation to optimize cluster orbitals
    val, vec = eigen(s)
    display(vec)
    values = Diagonal(val.^(-1/2))
    display(values)
    S_one_half = vec'*(values)*vec
    display(S_one_half)
    Da = S_one_half*Da_old*S_one_half
    Db = S_one_half*Db_old*S_one_half
    
    e_cmf, U, Da, Db = FermiCG.cmf_oo(ints, clusters, init_fspace, Da, Db, max_iter_oo=100, verbose=0, gconv=1e-6, method="cg", sequential=true);
    #e_cmf, U, Da, Db = FermiCG.cmf_oo(ints, clusters, init_fspace, rdm1, rdm1, max_iter_oo=100, verbose=0, gconv=1e-6, method="cg", sequential=true);
    #FermiCG.pyscf_write_molden(mol,lo_ao*U, filename="cmf.molden");
    #error("stop here")

    #rotate the integrals by the cmf calculation
    ints = FermiCG.orbital_rotation(ints, U);
    max_roots = 40

    #Build Cluster Basis (delta n is here)
    cluster_bases = FermiCG.compute_cluster_eigenbasis(ints, clusters, delta_elec=2, verbose=1, max_roots=max_roots, init_fspace=init_fspace, rdm1a=Da, rdm1b=Db);

    #@save "scan_after_cmf.jld2" ints Da Db e_cmf cluster_bases clusters init_fspace

    #Build Clustered Operator
    cluster_ham = FermiCG.extract_ClusteredTerms(ints, clusters);
    
    #Build Cluster Operators
    cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);
    FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, Da, Db);
    
    C_old = U
    Da_old = Da
    Db_old = Db
    
    #Need to find reference state 
    ref_fock = FermiCG.FockConfig(init_fspace)
    nroots = 13
    #ci_vector = FermiCG.TPSCIstate(clusters, ref_fock, R=nroots)
    ci_vector = FermiCG.TPSCIstate(clusters, FermiCG.FockConfig(init_fspace), R=nroots);
    #ci_vector = FermiCG.ClusteredState(clusters, ref_fock, R=nroots);
    #Need to find the automated way to define these other excited configs away from ref state, example is to large
    #to do by hand
    #probably something to do with building p spaces and q spaces
    
    ci_vector[ref_fock][ClusterConfig([1,1,1,1,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([2,1,1,1,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,2,1,1,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,1,2,1,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,1,1,2,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,1,1,1,2,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,1,1,1,1,2])] = zeros(Float64,nroots)
    
    ci_vector[ref_fock][ClusterConfig([3,1,1,1,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,3,1,1,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,1,3,1,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,1,1,3,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,1,1,1,3,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,1,1,1,1,3])] = zeros(Float64,nroots)
    
    #ci_vector[ref_fock][ClusterConfig([4,1,1,1,1,1])] = zeros(Float64,nroots)
    #ci_vector[ref_fock][ClusterConfig([1,4,1,1,1,1])] = zeros(Float64,nroots)
    #ci_vector[ref_fock][ClusterConfig([1,1,4,1,1,1])] = zeros(Float64,nroots)
    #ci_vector[ref_fock][ClusterConfig([1,1,1,4,1,1])] = zeros(Float64,nroots)
    #ci_vector[ref_fock][ClusterConfig([1,1,1,1,4,1])] = zeros(Float64,nroots)
    #ci_vector[ref_fock][ClusterConfig([1,1,1,1,1,4])] = zeros(Float64,nroots)

    FermiCG.eye!(ci_vector)
    display(ci_vector)

    #for thresh_cipsi in thresh_list
    e0, v0 = FermiCG.tpsci_ci(ci_vector, cluster_ops, cluster_ham,
                              thresh_cipsi=1e-3, # Threshold for adding to P-space
                              #thresh_cipsi=thresh_cipsi, # Threshold for adding to P-space
                              thresh_foi=1e-2,    # Threshold for keeping terms when defining FOIS
                              thresh_asci=1e-1,     # Threshold of P-space configs to search from
                              max_iter=10);

    @time e2 = FermiCG.compute_pt2_energy(v0, cluster_ops, cluster_ham, thresh_foi=1e-8)
    #name = "eq_tpsci_results"*string(0.001)*".jld2"
    #@save name e0 e2 v0 ecore
    
    println()
    println("	*======TPSCI results======*")
    @printf("TCI Thresh: %8.6f  Dim:%8d\n",1e-2,size(v0)[1])
    println()
    @printf("TCI %5s %12s %12s\n", "Root", "E(0)", "E(2)") 
    for r in 1:nroots
        @printf("TCI %5s %12.8f %12.8f\n",r, e0[r] + ecore, e0[r] + e2[r] + ecore)
        display(v0,thresh=1e-4,root=r)
    end
    #end
    
    push!(energies_ground, e0[1]+ecore)
    push!(energies_t1, e0[2]+ecore)
    push!(energies_t2, e0[3]+ecore)
    push!(energies_t3, e0[4]+ecore)
    push!(energies_t4, e0[5]+ecore)
    push!(energies_t5, e0[6]+ecore)
    push!(energies_t6, e0[7]+ecore)
    
    push!(energies_s1, e0[8]+ecore)
    push!(energies_s2, e0[9]+ecore)
    push!(energies_s3, e0[10]+ecore)
    push!(energies_s4, e0[11]+ecore)
    push!(energies_s5, e0[12]+ecore)
    push!(energies_s6, e0[13]+ecore)

    #push!(energies_t7, e0[14]+ecore)
    #push!(energies_t8, e0[15]+ecore)
    #push!(energies_t9, e0[16]+ecore)
    #push!(energies_t10, e0[17]+ecore)
    #push!(energies_t11, e0[18]+ecore)
    #push!(energies_t12, e0[19]+ecore)

    push!(pt2_energies, e2)
end

Parsing fcidump.he06_oct
converged SCF energy = -17.1342346844178
size of Lowdin ortho AO's:(54, 54)
    6

    He       0.0000000000000000       0.0000000000000000       0.0000000000000000 
    He       3.7038926571428572       0.0000000000000000       0.0000000000000000 
    He       0.0000000000000000       3.7038926571428572       0.0000000000000000 
    He       3.7038926571428572       3.7038926571428572       0.0000000000000000 
    He       1.8519463285714286       1.8519463285714286       2.6190476190476191 
    He       1.8519463285714286       1.8519463285714286      -2.6190476190476191 

Parsing fcidump.he06_oct
converged SCF energy = -17.1342346844178
size of Lowdin ortho AO's:(54, 54)


6-element Vector{Cluster}:
 Cluster(1, [1, 2, 3, 4, 5, 6, 7, 8, 9])
 Cluster(2, [10, 11, 12, 13, 14, 15, 16, 17, 18])
 Cluster(3, [19, 20, 21, 22, 23, 24, 25, 26, 27])
 Cluster(4, [28, 29, 30, 31, 32, 33, 34, 35, 36])
 Cluster(5, [37, 38, 39, 40, 41, 42, 43, 44, 45])
 Cluster(6, [46, 47, 48, 49, 50, 51, 52, 53, 54])

(54, 54)

54×54 Matrix{Float64}:
  1.88352      -1.70965       0.741502     …   0.0233536    -0.028656
 -1.70965       3.67302      -2.29327         -0.073919      0.0898855
  0.741502     -2.29327       2.64683          0.0862341    -0.102638
 -0.004798      0.0168506    -0.0241663       -0.0010135     0.00372761
 -0.004798      0.0168506    -0.0241663       -0.00478351    0.00372761
 -3.62743e-16   3.8709e-16   -1.21766e-16  …   0.00444539   -0.00572739
  0.0123907    -0.0436837     0.0630515        0.00107314   -0.00773683
  0.0123907    -0.0436837     0.0630515        0.0115752    -0.00773683
 -2.23213e-17  -3.97338e-16   4.9375e-16      -0.00964996    0.0126484
 -0.00922486    0.0400108    -0.0743503        0.0233536    -0.028656
  0.0400108    -0.158532      0.26747      …  -0.073919      0.0898855
 -0.0743503     0.26747      -0.398882         0.0862341    -0.102638
 -0.0159745     0.0508596    -0.0593676        0.0010135    -0.00372761
  ⋮                                        ⋱        

54×54 Matrix{Float64}:
  0.161718      5.71161e-6    0.0123704    …  -0.0947921     0.164187
  0.235074      7.23953e-6    0.0156796        0.203541     -0.352549
  0.288247      6.09106e-6    0.0131922       -0.179813      0.311451
  0.0124523    -1.40954e-7   -0.014874         0.00703881   -0.0121918
  0.0124523     1.34351e-7    0.0148597        0.00703881   -0.0121918
  2.42955e-16  -0.021037      2.05784e-7   …   1.49451e-16  -1.33314e-16
  0.0304774    -2.77223e-7   -0.0293198       -0.0155699     0.0269682
  0.0304774     2.65488e-7    0.0292944       -0.0155699     0.0269682
  1.44844e-16  -0.0414704     4.05664e-7      -5.54693e-17   5.30431e-17
  0.161718     -3.36708e-6   -0.363653        -0.0947943    -0.164186
  0.235074     -4.26781e-6   -0.460934     …   0.203546      0.352547
  0.288247     -3.59077e-6   -0.387812        -0.179817     -0.311448
 -0.0124523    -2.35447e-7   -0.000715906     -0.00703898   -0.0121917
  ⋮                                        ⋱            

...

54×54 Matrix{Float64}:
  1.25101      -0.548978      0.132139     …   0.00368055   -0.00452015
 -0.54855       1.69396      -0.700001        -0.016887      0.0207573
  0.130779     -0.696812      1.43405          0.03064      -0.0376883
 -0.000668957   0.00296358   -0.0050323       -0.000524701   0.00118708
 -0.000668957   0.00296358   -0.0050323       -0.00124164    0.00118708
 -9.49267e-14  -7.96034e-14  -1.71774e-14  …   0.00140274   -0.00173079
  0.00172285   -0.00807458    0.0145811        0.00137069   -0.00332795
  0.00172285   -0.00807458    0.0145811        0.00368048   -0.00332795
  1.30913e-13   1.19191e-13  -6.36514e-15     -0.00404588    0.00507142
 -0.00191379    0.00847043   -0.0140732        0.00368055   -0.00452015
  0.0032861    -0.0279243     0.0637714    …  -0.016887      0.0207573
 -0.00321998    0.0410352    -0.113739         0.03064      -0.0376883
 -0.0033369     0.0118074    -0.0148383        0.000524701  -0.00118708
  ⋮                                        ⋱ 

54×54 Matrix{ComplexF64}:
    -0.159518+0.0im     0.0476865+0.0im  …     -0.170052+0.0im
    -0.231948+0.0im     0.0604277+0.0im         0.358188+0.0im
    -0.293186+0.0im     0.0506532+0.0im         -0.30126+0.0im
  -0.00981167+0.0im    0.00874433+0.0im         0.012551+0.0im
  -0.00981167+0.0im   -0.00870665+0.0im         0.012551+0.0im
  1.70099e-11+0.0im     0.0103818+0.0im  …   2.03378e-14+0.0im
   -0.0252411+0.0im     0.0175405+0.0im       -0.0269288+0.0im
   -0.0252411+0.0im    -0.0175406+0.0im       -0.0269288+0.0im
  4.28983e-11+0.0im     0.0208702+0.0im     -5.92911e-14+0.0im
    -0.159518+0.0im      0.276866+0.0im         0.170229+0.0im
    -0.231948+0.0im      0.350841+0.0im  …     -0.358561+0.0im
    -0.293186+0.0im      0.294091+0.0im         0.301574+0.0im
   0.00981167+0.0im   -0.00161221+0.0im        0.0125641+0.0im
             ⋮                           ⋱  
 -2.24818e-12+0.0im     0.0145194+0.0im      8.47485e-14+0.0im
 -8.36482e-11+0.0im    -0.0205617+0.0im      1.

54×54 Diagonal{ComplexF64, Vector{ComplexF64}}:
 0.799097+0.0im           ⋅      …          ⋅              ⋅    
          ⋅      0.831238+0.0im             ⋅              ⋅    
          ⋅               ⋅                 ⋅              ⋅    
          ⋅               ⋅                 ⋅              ⋅    
          ⋅               ⋅                 ⋅              ⋅    
          ⋅               ⋅      …          ⋅              ⋅    
          ⋅               ⋅                 ⋅              ⋅    
          ⋅               ⋅                 ⋅              ⋅    
          ⋅               ⋅                 ⋅              ⋅    
          ⋅               ⋅                 ⋅              ⋅    
          ⋅               ⋅      …          ⋅              ⋅    
          ⋅               ⋅                 ⋅              ⋅    
          ⋅               ⋅                 ⋅              ⋅    
         ⋮                       ⋱                 
          ⋅               ⋅                 ⋅          

54×54 Matrix{ComplexF64}:
      1.01937-2.17805e-11im  …    0.00260879-2.25713e-11im
     0.012801+4.10335e-12im      0.000394799+4.05653e-12im
   -0.0566824+0.00107169im       -0.00241421+3.18383e-5im
   -0.0566824-0.00107169im       -0.00241421-3.18383e-5im
    0.0249652+0.0703879im          0.0108002-0.0042816im
    0.0249652-0.0703879im    …     0.0108002+0.0042816im
   -0.0278037-7.531e-13im         0.00270755-1.24952e-12im
   0.00155308+5.07969e-6im       -0.00320814-0.00184608im
   0.00155308-5.07969e-6im       -0.00320814+0.00184608im
   0.00725644-9.53568e-15im     -0.000189326-1.83919e-14im
 -0.000449465+1.02219e-12im  …   0.000459102-1.29107e-12im
   1.80813e-6-2.44126e-6im       -1.84692e-6+2.49361e-6im
   1.80813e-6+2.44126e-6im       -1.84692e-6-2.49361e-6im
             ⋮               ⋱  
 -0.000425214+1.00765e-12im      0.000487011-1.3055e-12im
   0.00355399+1.56812e-13im      -5.03774e-5+1.47977e-13im
   0.00476459-0.00324194im       -0.00202408+0.000539368im
   0.004

LoadError: InexactError: Float64(1.4432187582118536 + 2.6317140230501457e-11im)

In [ ]:
@save "scan_energies_triplet.jld2" energies_ground energies_t1 energies_t2 energies_t3 energies_t4 energies_t5 energies_t6
@save "scan_energies_singlet.jld2" energies_s1 energies_s2 energies_s3 energies_s4 energies_s5 energies_s6
#@save "scan_energies_second_triple.jld2" energies_t7 energies_t8 energies_t9 energies_t10 energies_t11 energies_t12
@save "pt2_energies.jld2" pt2_energies